# Computer Vision Final Project
## Spring 2022
### Matthew Menzi

Report and presentation at:
https://drive.google.com/drive/folders/1OhXLYOmTM7ldXHgdVLO4I9yEmVxEkUIy?usp=sharing

The first portion of the notebook is installations, data loading, and helper functions. Second section is where more substantial code is.

In [ ]:
# Installations 

# !pip install kornia
!pip install kornia_moons
# !pip install ../input/kornia-loftr/kornia-0.6.4-py2.py3-none-any.whl
!pip install git+https://github.com/kornia/kornia


In [ ]:
!cp -r ../input/imutils/imutils-0.5.3/ /
!pip install /imutils-0.5.3/

In [ ]:
# Imports

import cv2
import numpy as np
import os
import pandas as pd
import gc
import tensorflow as tf
import random
import kornia as K
import kornia.feature as KF
import gc
from kornia_moons.feature import *
import matplotlib.pyplot as plt
import torch
import pydegensac
from kornia.feature.loftr import LoFTR
from kornia.feature.hynet import HyNet
from kornia.feature.keynet import KeyNetDetector
import time
import math
from scipy.spatial import distance as dist
import imutils


In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    print ("GPU mode")
else:
    print ("CPU mode")

In [ ]:
# Data imports

DATA_DIR = "/kaggle/input/image-matching-challenge-2022"

TRAIN_DIR = os.path.join(DATA_DIR, "train")
SCALING_CSV = os.path.join(TRAIN_DIR, "scaling_factors.csv")
scaling_df = pd.read_csv(SCALING_CSV)
scale_map = scaling_df.groupby("scene")["scaling_factor"].first().to_dict()

TRAIN_SCENES = scaling_df.scene.unique().tolist()
train_map = {} 

for _s in TRAIN_SCENES:
    # Initialize    
    train_map[_s] = {}
    
    # Image Stuff
    train_map[_s]["images"] = sorted(tf.io.gfile.glob(os.path.join(TRAIN_DIR, _s, "images", "*.jpg")))
    train_map[_s]["image_ids"] = [_f_path[:-4].rsplit("/", 1)[-1] for _f_path in train_map[_s]["images"]]
    
    # Calibration Stuff (CAL)
    _tmp_cal_df = pd.read_csv(os.path.join(TRAIN_DIR, _s, "calibration.csv"))
    _tmp_cal_df["image_path"] = os.path.join(TRAIN_DIR, _s, "images")+"/"+_tmp_cal_df["image_id"]+".jpg"
    train_map[_s]["cal_df"]=_tmp_cal_df.copy()
        
    # Pair Covisibility Stuff (PCO)
    _tmp_pco_df = pd.read_csv(os.path.join(TRAIN_DIR, _s, "pair_covisibility.csv"))
    _tmp_pco_df["image_id_1"], _tmp_pco_df["image_id_2"] = zip(*_tmp_pco_df.pair.apply(lambda x: x.split("-")))
    _tmp_pco_df["image_path_1"] = os.path.join(TRAIN_DIR, _s, "images")+"/"+_tmp_pco_df["image_id_1"]+".jpg"
    _tmp_pco_df["image_path_2"] = os.path.join(TRAIN_DIR, _s, "images")+"/"+_tmp_pco_df["image_id_2"]+".jpg"
    train_map[_s]["pco_df"] = _tmp_pco_df.copy()

#cleanup
del _tmp_cal_df, _tmp_pco_df; gc.collect(); gc.collect();
torch.cuda.empty_cache()

_cal_dfs = []
_pco_dfs = []
_s_dfs = []
for _s in TRAIN_SCENES:
    _s_map = train_map[_s]
    _s_dfs.append(pd.DataFrame({
        "scene":[_s,]*len(_s_map["images"]),
        "f_path":_s_map["images"],
    }))
    _cal_dfs.append(_s_map["cal_df"])
    _pco_dfs.append(_s_map["pco_df"])

all_pco_dfs = pd.concat(_pco_dfs).reset_index(drop=True)
all_cal_dfs = pd.concat(_cal_dfs).reset_index(drop=True)

train_df = pd.concat(_s_dfs).reset_index(drop=True)
train_df.insert(0, "image_id", train_df.f_path.apply(lambda x: x[:-4].rsplit("/", 1)[-1]))
train_df = pd.merge(train_df, all_cal_dfs, on="image_id").drop(columns=["image_path",])

cov_1_df = all_pco_dfs[["image_id_1", "covisibility"]]
cov_1_df.columns = ["image_id", "covisibility"]
cov_2_df = all_pco_dfs[["image_id_2", "covisibility"]]
cov_2_df.columns = ["image_id", "covisibility"]
cov_df = pd.concat([cov_1_df,cov_2_df])
img_id_2_cov = cov_df.groupby("image_id")["covisibility"].mean().to_dict()
del cov_1_df, cov_2_df, cov_df; gc.collect(); torch.cuda.empty_cache()

# Add a column for average covisibility
#    - i.e. For a given image, what is the average 
#           covisibility of all of it's respective pairs
train_df.insert(2, "mean_covisibility", train_df.image_id.map(img_id_2_cov))

train_df["fx"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[0])) # 0 = TL 
train_df["fy"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[4])) # 4 = C
train_df["x0"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[2])) # 0 = TR
train_df["y0"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[5])) # 4 = CR
train_df["s"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[1])) # 1 = TC

if (train_df["fx"]!=train_df["fy"]).sum()==0:
    print("All fx values (focal length x) are equal to the respective fy values (focal length y)... as expected")
    
if train_df["s"].sum()==0:
    print("All skew values are 0... as expected")

train_df["t_x"] = train_df.translation_vector.apply(lambda x: float(x.split()[0]))
train_df["t_y"] = train_df.translation_vector.apply(lambda x: float(x.split()[1]))
train_df["t_z"] = train_df.translation_vector.apply(lambda x: float(x.split()[2]))

train_df["r1_1"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[0]))
train_df["r2_1"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[1]))
train_df["r3_1"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[2]))
train_df["r1_2"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[3]))
train_df["r2_2"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[4]))
train_df["r3_2"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[5]))
train_df["r1_3"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[6]))
train_df["r2_3"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[7]))
train_df["r3_3"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[8]))

print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")


In [ ]:
# Convenience function to stack two images with different sizes.
def build_composite_image(im1, im2, axis=1, margin=0, background=1):
    """
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """
    
    if background != 0 and background != 1:
        background = 1
    if axis != 0 and axis != 1:
        raise RuntimeError('Axis must be 0 (vertical) or 1 (horizontal')

    h1, w1, _ = im1.shape
    h2, w2, _ = im2.shape

    if axis == 1:
        composite = np.zeros((max(h1, h2), w1 + w2 + margin, 3), dtype=np.uint8) + 255 * background
        if h1 > h2:
            voff1, voff2 = 0, (h1 - h2) // 2
        else:
            voff1, voff2 = (h2 - h1) // 2, 0
        hoff1, hoff2 = 0, w1 + margin
    else:
        composite = np.zeros((h1 + h2 + margin, max(w1, w2), 3), dtype=np.uint8) + 255 * background
        if w1 > w2:
            hoff1, hoff2 = 0, (w1 - w2) // 2
        else:
            hoff1, hoff2 = (w2 - w1) // 2, 0
        voff1, voff2 = 0, h1 + margin
    composite[voff1:voff1 + h1, hoff1:hoff1 + w1, :] = im1
    composite[voff2:voff2 + h2, hoff2:hoff2 + w2, :] = im2

    return (composite, (voff1, voff2), (hoff1, hoff2))

#  Draw keypoints and matches.
def draw_cv_matches(im1, im2, kp1, kp2, matches, axis=1, margin=0, background=0, linewidth=2):
    """
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """
    
    composite, v_offset, h_offset = build_composite_image(im1, im2, axis, margin, background)

    # Draw all keypoints.
    for coord_a, coord_b in zip(kp1, kp2):
        composite = cv2.drawMarker(composite, (int(coord_a[0] + h_offset[0]), int(coord_a[1] + v_offset[0])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
        composite = cv2.drawMarker(composite, (int(coord_b[0] + h_offset[1]), int(coord_b[1] + v_offset[1])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
    
    # Draw matches, and highlight keypoints used in matches.
    for idx_a, idx_b in matches:
        composite = cv2.drawMarker(composite, (int(kp1[idx_a, 0] + h_offset[0]), int(kp1[idx_a, 1] + v_offset[0])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.drawMarker(composite, (int(kp2[idx_b, 0] + h_offset[1]), int(kp2[idx_b, 1] + v_offset[1])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.line(composite,
                             tuple([int(kp1[idx_a][0] + h_offset[0]),
                                   int(kp1[idx_a][1] + v_offset[0])]),
                             tuple([int(kp2[idx_b][0] + h_offset[1]),
                                   int(kp2[idx_b][1] + v_offset[1])]), color=(0, 0, 255), thickness=1)
    return composite

# K = calibration matrix
def normalize_keypoints(keypoints, K):
    C_x = K[0, 2]
    C_y = K[1, 2]
    f_x = K[0, 0]
    f_y = K[1, 1]
    keypoints = (keypoints - np.array([[C_x, C_y]])) / np.array([[f_x, f_y]])
    return keypoints

# F = fundamental matrix K1, K2 = calibration matrices kp1, kp2 = keypoints
def compute_essential_matrix(F, K1, K2, kp1, kp2):
    """
    Compute the Essential matrix from the Fundamental matrix, given the calibration matrices. 
    Note that we ask participants to estimate F, i.e., without relying on known intrinsics.
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """

    assert F.shape[0] == 3, 'Malformed F?'

    # Use OpenCV's recoverPose to solve the cheirality check:
    # https://docs.opencv.org/4.5.4/d9/d0c/group__calib3d.html#gadb7d2dfcc184c1d2f496d8639f4371c0
    E = np.matmul(np.matmul(K2.T, F), K1).astype(np.float64)
    
    kp1n = normalize_keypoints(kp1, K1)
    kp2n = normalize_keypoints(kp2, K2)
    num_inliers, R, T, mask = cv2.recoverPose(E, kp1n, kp2n)

    return E, R, T

def quaternion_from_matrix(matrix):
    """
    Transform a rotation matrix into a quaternion
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    
    The quaternion is the eigenvector of K that corresponds to the largest eigenvalue.\
    """

    M = np.array(matrix, dtype=np.float64, copy=False)[:4, :4]
    m00 = M[0, 0]
    m01 = M[0, 1]
    m02 = M[0, 2]
    m10 = M[1, 0]
    m11 = M[1, 1]
    m12 = M[1, 2]
    m20 = M[2, 0]
    m21 = M[2, 1]
    m22 = M[2, 2]

    K = np.array([
        [m00 - m11 - m22, 0.0, 0.0, 0.0],
        [m01 + m10, m11 - m00 - m22, 0.0, 0.0],
        [m02 + m20, m12 + m21, m22 - m00 - m11, 0.0],
        [m21 - m12, m02 - m20, m10 - m01, m00 + m11 + m22]
    ])
    K /= 3.0

    # The quaternion is the eigenvector of K that corresponds to the largest eigenvalue.
    w, V = np.linalg.eigh(K)
    q = V[[3, 0, 1, 2], np.argmax(w)]
    if q[0] < 0:
        np.negative(q, q)
    return q

def compute_error_for_example(q_gt, T_gt, q, T, scale, eps=1e-15):
    '''Compute the error metric for a single example.
    
    The function returns two errors, over rotation and translation. 
    These are combined at different thresholds by ComputeMaa in order to compute the mean Average Accuracy.'''
    
    q_gt_norm = q_gt / (np.linalg.norm(q_gt) + eps)
    q_norm = q / (np.linalg.norm(q) + eps)

    loss_q = np.maximum(eps, (1.0 - np.sum(q_norm * q_gt_norm)**2))
    err_q = np.arccos(1 - 2 * loss_q)

    # Apply the scaling factor for this scene.
    T_gt_scaled = T_gt * scale
    T_scaled = T * np.linalg.norm(T_gt) * scale / (np.linalg.norm(T) + eps)

    err_t = min(np.linalg.norm(T_gt_scaled - T_scaled), np.linalg.norm(T_gt_scaled + T_scaled))

    return err_q * 180 / np.pi, err_t

# Compute mean average error
def ComputeMaa(err_q, err_t, thresholds_q, thresholds_t):
    '''Compute the mean Average Accuracy at different tresholds, for one scene.'''
    
    assert len(err_q) == len(err_t)
    
    acc, acc_q, acc_t = [], [], []
    for th_q, th_t in zip(thresholds_q, thresholds_t):
        acc += [(np.bitwise_and(np.array(err_q) < th_q, np.array(err_t) < th_t)).sum() / len(err_q)]
        acc_q += [(np.array(err_q) < th_q).sum() / len(err_q)]
        acc_t += [(np.array(err_t) < th_t).sum() / len(err_t)]
    return np.mean(acc), np.array(acc), np.array(acc_q), np.array(acc_t)

def resize_keep_ratio(img, longest_size=1500):
    scale = 840 / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    resized_img = cv2.resize(img, (w, h))
    return resized_img

def resize(img, target_size=(640,480)):
    resized_img = cv2.resize(img, target_size)
    return resized_img

def load_torch_image(fname):
    img = cv2.imread(fname)
    img = resize_keep_ratio(img)
#     img = cv2.resize(img, (img.shape[1]//8*8, img.shape[0]//8*8))  # input size should be divisible by 8
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img

#  Draw keypoints and matches.
def draw_cv_matches(im1, im2, kp1, kp2, matches, axis=1, margin=0, background=0, linewidth=2):
    """
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """
    
    composite, v_offset, h_offset = build_composite_image(im1, im2, axis, margin, background)

    # Draw all keypoints.
    for coord_a, coord_b in zip(kp1, kp2):
        composite = cv2.drawMarker(composite, (int(coord_a[0] + h_offset[0]), int(coord_a[1] + v_offset[0])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
        composite = cv2.drawMarker(composite, (int(coord_b[0] + h_offset[1]), int(coord_b[1] + v_offset[1])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
    
    # Draw matches, and highlight keypoints used in matches.
    for idx_a, idx_b in matches:
        composite = cv2.drawMarker(composite, (int(kp1[idx_a, 0] + h_offset[0]), int(kp1[idx_a, 1] + v_offset[0])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.drawMarker(composite, (int(kp2[idx_b, 0] + h_offset[1]), int(kp2[idx_b, 1] + v_offset[1])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.line(composite,
                             tuple([int(kp1[idx_a][0] + h_offset[0]),
                                   int(kp1[idx_a][1] + v_offset[0])]),
                             tuple([int(kp2[idx_b][0] + h_offset[1]),
                                   int(kp2[idx_b][1] + v_offset[1])]), color=(0, 0, 255), thickness=1)
    return composite

# Key Aspects of Project

The goal of this notebook is not to provide a working submission to the contest, but to explore the options that Kornia has to offer for solving the problem addressed by the contest. This notebook will test different ways to set up a correspondence-finding pipeline and use information gleaned from the experiment to hopefully optimize an algorithm to be used for a submission.

All the code above this point was provided by the contest organizers. All of the code after this point was written by myself (Matthew Menzi).

## 1. Candidate keypoint finder (e.g., SIFT)

Using Kornia, keypoints are "patches" of 32x32 pixels

### Options:
* SIFT basic keypoints
* KeyNetDetector(pretrained=False, num_features=2048, keynet_conf=keynet_config['KeyNet_default_config'], ori_module=PassLAF(), aff_module=PassLAF())
(Ended up not using KND because performance seemed worse at cursory glance and wasn't time to pursue it)

## 2. Keypoint descriptor

### Options:
* DenseSIFTDescriptor computes SIFT descriptor densely over image (no keypoint finder needed)
* SIFTDescriptor(patch_size=41, num_ang_bins=8, num_spatial_bins=4, rootsift=True, clipval=0.2) 
* MKDDescriptor(patch_size=32, kernel_type='concat', whitening='pcawt', training_batch='liberty', output_dims=128) Multiple Kernal location descriptors
* HardNet(pretrained=False) computes HardNet descriptors
* HardNet8 (better hardnet) 
* HyNet(pretrained=False, is_bias=True, is_bias_FRN=True, dim_desc=128, drop_rate=0.3, eps_l2_norm=1e-10) computes HyNet descriptors
* TFeat(pretrained=False)
* SOSNet(pretrained=False)
(Didn't use MKDDescriptor because it was hard on the GPU)

## 3. Keypoint matcher

### Options:
* match_nn(desc1, desc2, dm=None) finds nearest neighbors in desc2 for each vector in desc1
* match_mnn(desc1, desc2, dm=None) finds mutual nearest neighbors in desc2 for each vector in desc1
* match_snn(desc1, desc2, th=0.8, dm=None) The method satisfies first to second nearest neighbor distance <= th.
* match_smnn(desc1, desc2, th=0.8, dm=None) 
* LoFTR

## 4. Match selector
* cv2.FindFundamentalMat
* Performance for the cv2 MAGSAC option seems to be considered best


In [ ]:
# Show pair of images with matches before RANSAC and matches after RANSAC 
def display_example_from_batch(batch_df, loftr=False, resize=False):
    idx = random.randint(1, batch_df.shape[0])
    rowA = batch_df.iloc[idx]
    if loftr:
        centers1a = rowA['kp1']
        centers2a = rowA['kp2']
    else:
        laf1a = rowA['kp1']
        laf2a = rowA['kp2']
        centers1a = KF.laf.get_laf_center(laf1a)[0]
        centers2a = KF.laf.get_laf_center(laf2a)[0]
    img1a = cv2.imread(rowA['img1'])
    img2a = cv2.imread(rowA['img2'])
    if resize:
        img1a, img2a = resize_keep_ratio(img1a), resize_keep_ratio(img2a)
    matches_beforeA = rowA['MatchIdxBefore']
    matches_afterA = rowA['MatchIdxAfter']
    beforeRansacA = draw_cv_matches(img1a, img2a, centers1a, centers2a, matches_beforeA)
    afterRansacA = draw_cv_matches(img1a, img2a, centers1a, centers2a, matches_afterA)
    rowB = batch_df.iloc[0]
    if loftr:
        centers1b = rowB['kp1']
        centers2b = rowB['kp2']
    else:
        laf1b = rowB['kp1']
        laf2b = rowB['kp2']
        centers1b = KF.laf.get_laf_center(laf1b)[0]
        centers2b = KF.laf.get_laf_center(laf2b)[0]
    img1b = cv2.imread(rowB['img1'])
    img2b = cv2.imread(rowB['img2'])
    if resize:
        img1b, img2b = resize_keep_ratio(img1b), resize_keep_ratio(img2b)
    matches_beforeB = rowB['MatchIdxBefore']
    matches_afterB = rowB['MatchIdxAfter']
    beforeRansacB = draw_cv_matches(img1b, img2b, centers1b, centers2b, matches_beforeB)
    afterRansacB = draw_cv_matches(img1b, img2b, centers1b, centers2b, matches_afterB)
    plt.figure(figsize=(20, 10))
    plt.subplot(2,2,1)
    plt.imshow(beforeRansacB)
    plt.subplot(2,2,3)
    plt.imshow(afterRansacB)
    plt.subplot(2,2,2)
    plt.imshow(beforeRansacA)
    plt.subplot(2,2,4)
    plt.imshow(afterRansacA)
    plt.tight_layout()
    plt.show()
    
    
    

In [ ]:
# Load batch of image pairs to use for testing
# All operations are done through a dataframe with pertinent info added as a column
# as process iterates through following functions

def arr_from_str(_s):
    return np.fromstring(_s, sep=" ").reshape((-1,3)).squeeze()

# Returns pair of images with GT fundamental matrix and covisibility
def get_pair(location, row_number):
    row = train_map[location]['pco_df'].iloc[row_number]
    path_1 = row.image_path_1
    path_2 = row.image_path_2
    covis = row.covisibility
    img1 = path_1
    img2 = path_2
    fund = arr_from_str(row.fundamental_matrix)
    return img1, img2, fund, covis

# Returns intrinsics and extrinsics for image pair
def get_pair_info(location, row_number):
    row = train_map[location]['pco_df'].iloc[row_number]
    img_id_1 = row.image_id_1
    img_id_2 = row.image_id_2
    r1 = train_df.loc[train_df['image_id'] == img_id_1]
    r2 = train_df.loc[train_df['image_id'] == img_id_2]
    ci_1 = r1.iloc[0]['camera_intrinsics']
    ci_2 = r2.iloc[0]['camera_intrinsics']
    rm_1 = r1.iloc[0]['rotation_matrix']
    rm_2 = r2.iloc[0]['rotation_matrix']
    tv_1 = r1.iloc[0]['translation_vector']
    tv_2 = r2.iloc[0]['translation_vector']
    return arr_from_str(ci_1), arr_from_str(ci_2), arr_from_str(rm_1), arr_from_str(rm_2), arr_from_str(tv_1).reshape((3,1)), arr_from_str(tv_2).reshape((3,1))

# Returns a batch of image pairs with n_per_loc images per location (min 16 images in batch) with min_covis minimum covisibility
def get_pair_batch(n_per_loc, min_covis=0.1):
    dat = []
    locs = scale_map.keys()
    for loc in locs:
        n_rows = train_map[loc]['pco_df'].loc[train_map[loc]['pco_df']['covisibility'] > min_covis].shape[0]
        for i in range(n_per_loc):
            row = random.randint(0, n_rows)
            img1, img2, F, covis = get_pair(loc, row)
            ci_1, ci_2, rm_1, rm_2, tv_1, tv_2 = get_pair_info(loc, row)
            scale_factor = scale_map[loc]
            dat.append([loc, scale_factor, covis, img1, img2, F, ci_1, ci_2, rm_1, rm_2, tv_1, tv_2])
    batch_df = pd.DataFrame(dat, columns=['scene', 'scale', 'covis', 'img1', 'img2', 'F', 'ci_1', 'ci_2', 'rm_1', 'rm_2', 'tv_1', 'tv_2'])
    del dat; gc.collect()
    torch.cuda.empty_cache()
    return batch_df


In [ ]:
# Get candidate lafs (keypoints) for image pairs
def extract_sift_lafs(rgb_image):
    n_feat = 8_000
    contrast_thresh = -10_000
    edge_thresh = -10_000
    
    sift_detector = cv2.SIFT_create(n_feat, contrastThreshold=contrast_thresh, edgeThreshold=edge_thresh)
    
    gray = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)

    keypoints = sift_detector.detect(gray)
    
    lafs = laf_from_opencv_SIFT_kpts(keypoints)
    
    return lafs

# Get KeyNet candidate keypoints
def extract_KeyNet_keypoints(rgb_image, pre_tr):
    with torch.no_grad():
        detector = KF.KeyNetDetector(pretrained=pre_tr).eval().to(device)
        gray = K.color.rgb_to_grayscale(K.image_to_tensor(rgb_image, False).float()/255.).to(device)
        that = detector.forward(gray)[0]
    return that

# Get LAFs for batch of image pairs 
def get_batch_lafs(batch_df, SIFT=True, resize=False):
    keypoints1 = []
    keypoints2 = []
    for i in range(batch_df.shape[0]):
        row = batch_df.iloc[i]
        if SIFT:
            img1, img2 = cv2.imread(row['img1']), cv2.imread(row['img2'])
            if resize:
                img1 = resize_keep_ratio(img1)
                img2 = resize_keep_ratio(img2)
            kp1 = extract_sift_lafs(img1)
            kp2 = extract_sift_lafs(img2)
            keypoints1.append(kp1.cpu())
            keypoints2.append(kp2.cpu())
            del img1, img2, kp1, kp2; gc.collect(), gc.collect(); torch.cuda.empty_cache()
        else:
            img1, img2 = cv2.imread(row['img1']), cv2.imread(row['img2'])
            if resize:
                img1 = resize_keep_ratio(img1)
                img2 = resize_keep_ratio(img2)
            kp1 = extract_KeyNet_keypoints(img1, True)
            kp2 = extract_KeyNet_keypoints(img2, True)
            keypoints1.append(kp1.cpu())
            keypoints2.append(kp2.cpu())
            del img1, img2, kp1, kp2; gc.collect(), gc.collect(); torch.cuda.empty_cache()
        del row
        gc.collect()
        torch.cuda.empty_cache()

    new_batch_df = batch_df.copy()
    new_batch_df['kp1'] = keypoints1
    new_batch_df['kp2'] = keypoints2
    new_batch_df['resized'] = [resize for i in range(batch_df.shape[0])]
    return new_batch_df

# Get descriptors for batch of image pairs
def get_batch_descriptors(batch_df, kornia_descriptor, resize=False):
    desc1 = []
    desc2 = []
    times = []
    with torch.no_grad():
        kornia_descriptor.eval()
        for i in range(batch_df.shape[0]):
            t = time.process_time()
            #print(f'Getting descriptors: iteration {i}')
            row = batch_df.iloc[i]
            img1, img2 = cv2.imread(row['img1']), cv2.imread(row['img2'])
            if resize:
                img1 = resize_keep_ratio(img1)
                img2 = resize_keep_ratio(img2)
            timg1 = K.color.rgb_to_grayscale(K.image_to_tensor(img1, False).float()/255.).to(device)
            timg2 = K.color.rgb_to_grayscale(K.image_to_tensor(img2, False).float()/255.).to(device)
            laf1 = row['kp1']
            laf2 = row['kp2']
            laf1, laf2 = laf1.to(device), laf2.to(device)
            descs1 = KF.get_laf_descriptors(timg1, laf1, kornia_descriptor)
            descs2 = KF.get_laf_descriptors(timg2, laf2, kornia_descriptor)
            desc1.append(descs1.cpu())
            desc2.append(descs2.cpu())
            del img1, img2, timg1, timg2, row, descs1, descs2; gc.collect(); torch.cuda.empty_cache()
            elapsed_time = time.process_time() - t
            times.append(elapsed_time)
    new_batch_df = batch_df.copy()
    new_batch_df['descs1'] = desc1
    new_batch_df['descs2'] = desc2
    new_batch_df['desc_time'] = times
    return new_batch_df

# Get matches for batch of image pairs
def get_batch_matches(batch_df):
    kp1before = []
    kp2before = []
    kp1after = []
    kp2after = []
    match_idx_before = []
    match_idx_after = []
    Fs = []
    times = []
    for i in range(batch_df.shape[0]):
        t = time.process_time()
        #print(f'Getting matches: iteration {i}')
        row = batch_df.iloc[i]
        desc1, desc2 = row['descs1'][0], row['descs2'][0]
        laf1, laf2 = row['kp1'], row['kp2']
        
        dists, idxs = KF.match_smnn(desc1, desc2, 0.95)
        
        cv_matches = cv2_matches_from_kornia(dists, idxs)
        centers1 = KF.laf.get_laf_center(laf1)[0]
        centers2 = KF.laf.get_laf_center(laf2)[0]
        kp_1_pos = np.float32([ list(centers1[m.queryIdx]) for m in cv_matches ])
        kp_2_pos = np.float32([ list(centers2[m.trainIdx]) for m in cv_matches ])
        matches_before = np.array([[m.queryIdx, m.trainIdx] for m in cv_matches])
        F, inlier_mask = cv2.findFundamentalMat(kp_1_pos, kp_2_pos, 
                                    cv2.USAC_MAGSAC, ransacReprojThreshold=0.25, 
                                    confidence=0.99999, maxIters=10000)
        matches_after = np.array([match for match, is_inlier in zip(matches_before, inlier_mask) if is_inlier])
        kp_1_after = np.float32([ list(centers1[m[0]]) for m in matches_after])
        kp_2_after = np.float32([ list(centers2[m[1]]) for m in matches_after])
        Fs.append(F)
        kp1before.append(kp_1_pos)
        kp2before.append(kp_2_pos)
        kp1after.append(kp_1_after)
        kp2after.append(kp_2_after)
        match_idx_before.append(matches_before)
        match_idx_after.append(matches_after)
        del row, desc1, desc2, laf1, laf2, dists, idxs, cv_matches, kp_1_pos, kp_2_pos, kp_1_after, kp_2_after, matches_before, F, inlier_mask, matches_after; 
        gc.collect(), torch.cuda.empty_cache()
        elapsed_time = time.process_time() - t
        times.append(elapsed_time)
    new_batch_df = batch_df.copy()
    new_batch_df['MatchesBefore1'] = kp1before
    new_batch_df['MatchesBefore2'] = kp2before
    new_batch_df['MatchesAfter1'] = kp1after
    new_batch_df['MatchesAfter2'] = kp2after
    new_batch_df['MatchIdxBefore'] = match_idx_before
    new_batch_df['MatchIdxAfter'] = match_idx_after
    new_batch_df['FoundF'] = Fs
    new_batch_df['match_time'] = times
    return new_batch_df


In [ ]:
# LoFTR matcher
WEIGHT_PATH_A = '../input/kornia-loftr/outdoor_ds.ckpt'
WEIGHT_PATH_B = '../input/kornia-loftr/loftr_outdoor.ckpt'

# LONGEST_EDGE = 1500
LOFTR_A = LoFTR(pretrained=None)
LOFTR_A.load_state_dict(torch.load(WEIGHT_PATH_A)['state_dict'])
LOFTR_A = LOFTR_A.to(device)
LOFTR_A.eval()

LOFTR_B = LoFTR(pretrained=None)
LOFTR_B.load_state_dict(torch.load(WEIGHT_PATH_B)['state_dict'])
LOFTR_B = LOFTR_B.to(device)
LOFTR_B.eval()

# Function for LoFTR model, which bypasses the descriptor stage of the process
def loftr_match(batch_df, matcher, resize=False):
    kp1before = []
    kp2before = []
    kp1after = []
    kp2after = []
    match_idx_before = []
    match_idx_after = []
    Fs = []
    times = []
    with torch.no_grad():
        for i in range(batch_df.shape[0]):
            t = time.process_time()
            row = batch_df.iloc[i]
            img1 = cv2.imread(row['img1'])
            img2 = cv2.imread(row['img2'])
            if resize:
                img1 = resize_keep_ratio(img1)
                img2 = resize_keep_ratio(img2)
            img1 = K.color.rgb_to_grayscale(K.image_to_tensor(img1, False).float()/255.).to(device)
            img2 = K.color.rgb_to_grayscale(K.image_to_tensor(img2, False).float()/255.).to(device)
            input_dict = {"image0": img1, "image1": img2}
            out = matcher(input_dict)
            kp_1_before = out['keypoints0']
            kp_2_before = out['keypoints1']
            indexes = out['batch_indexes']
            kp_1_before = kp_1_before.cpu().numpy()
            kp_2_before = kp_2_before.cpu().numpy()
            F, inlier_mask = cv2.findFundamentalMat(kp_1_before, kp_2_before, cv2.USAC_MAGSAC, 0.25, 0.999, 100000)
            assert F.shape == (3, 3), 'Malformed F?'
            kp_1_after = kp_1_before[inlier_mask.reshape(-1) > 0]
            kp_2_after = kp_2_before[inlier_mask.reshape(-1) > 0]
            matches_before = np.array([[i, i] for i in range(len(kp_1_before))])
            matches_after = np.array([match for match, is_inlier in zip(matches_before, inlier_mask) if is_inlier])
            Fs.append(F)
            kp1before.append(kp_1_before)
            kp2before.append(kp_2_before)
            kp1after.append(kp_1_after)
            kp2after.append(kp_2_after)
            match_idx_before.append(matches_before)
            match_idx_after.append(matches_after)
            del row, img1, img2, input_dict, out, kp_1_before, kp_2_before, kp_1_after, kp_2_after, matches_before, F, inlier_mask, matches_after; 
            gc.collect(),torch.cuda.empty_cache()
            elapsed_time = time.process_time() - t
            times.append(elapsed_time)
    new_batch_df = batch_df.copy()
    new_batch_df['kp1'] = kp1before
    new_batch_df['kp2'] = kp2before
    new_batch_df['descs1'] = [0 for i in range(batch_df.shape[0])]
    new_batch_df['descs2'] = [0 for i in range(batch_df.shape[0])]
    new_batch_df['desc_time'] = [0 for i in range(batch_df.shape[0])]
    new_batch_df['MatchesBefore1'] = kp1before
    new_batch_df['MatchesBefore2'] = kp2before
    new_batch_df['MatchesAfter1'] = kp1after
    new_batch_df['MatchesAfter2'] = kp2after
    new_batch_df['MatchIdxBefore'] = match_idx_before
    new_batch_df['MatchIdxAfter'] = match_idx_after
    new_batch_df['FoundF'] = Fs
    new_batch_df['match_time'] = times
    new_batch_df['resized'] = [resize for i in range(batch_df.shape[0])]
    return new_batch_df

In [ ]:
# Returns Mean Average Accuracy for batch 
def get_score(batch_df):
    errors_q = []
    errors_t = []
    thresholds_q = np.linspace(1, 10, 10)
    thresholds_t = np.geomspace(0.2, 5, 10)
        
    for i in range(batch_df.shape[0]):
        row = batch_df.iloc[i]
        SCALE_FACTOR = row['scale']
        inlier_kp_1 = np.array([row['MatchesAfter1']])
        inlier_kp_2 = np.array([row['MatchesAfter2']])
        F = row['FoundF']
        E, R, T = compute_essential_matrix(F, row['ci_1'], row['ci_2'], inlier_kp_1, inlier_kp_2)
        q = quaternion_from_matrix(R)
        T = T.flatten()
        R1_gt = row['rm_1']
        R2_gt = row['rm_2']
        T1_gt = row['tv_1']
        T2_gt = row['tv_2']
        dR_gt = np.dot(R2_gt, R1_gt.T)
        dT_gt = (T2_gt - np.dot(dR_gt, T1_gt)).flatten()
        q_gt = quaternion_from_matrix(dR_gt)
        q_gt = q_gt / (np.linalg.norm(q_gt) + 1e-15)
        err_q, err_t = compute_error_for_example(q_gt, dT_gt, q, T, SCALE_FACTOR)
        errors_q.append(err_q)
        errors_t.append(err_t)
    maa, overall, rotation, translation = ComputeMaa(errors_q, errors_t, thresholds_q, thresholds_t)
    print(f'Mean average accuracy: {maa}')

# Returns MAA, rotation error, and translation error for a row
def get_row_score(row):
    errors_q = []
    errors_t = []
    thresholds_q = np.linspace(1, 10, 10)
    thresholds_t = np.geomspace(0.2, 5, 10)
    SCALE_FACTOR = row['scale']
    inlier_kp_1 = np.array([row['MatchesAfter1']])
    inlier_kp_2 = np.array([row['MatchesAfter2']])
    F = row['FoundF']
    E, R, T = compute_essential_matrix(F, row['ci_1'], row['ci_2'], inlier_kp_1, inlier_kp_2)
    q = quaternion_from_matrix(R)
    T = T.flatten()
    R1_gt = row['rm_1']
    R2_gt = row['rm_2']
    T1_gt = row['tv_1']
    T2_gt = row['tv_2']
    dR_gt = np.dot(R2_gt, R1_gt.T)
    dT_gt = (T2_gt - np.dot(dR_gt, T1_gt)).flatten()
    q_gt = quaternion_from_matrix(dR_gt)
    q_gt = q_gt / (np.linalg.norm(q_gt) + 1e-15)
    err_q, err_t = compute_error_for_example(q_gt, dT_gt, q, T, SCALE_FACTOR)
    errors_q.append(err_q)
    errors_t.append(err_t)
    maa, overall, rotation, translation = ComputeMaa(errors_q, errors_t, thresholds_q, thresholds_t)
    return maa, err_q, err_t
    
# Organizes info from batch dataframe and writes it to CSV
# Pretty messy function but it gets the job done
# This data will be analyzed in a different location
def stats_to_csv(descriptorName, pre_trained, batch_df, overwrite=False):
    descriptor_name = []
    pretrained = []
    scene = []
    maa = []
    r_error = []
    t_error = []
    covis = []
    size_dif = []
    width1 = []
    width2 = []
    height1 = []
    height2 = []
    keypoints1 = []
    keypoints2 = []
    matches_before = []
    matches_after = []
    pct_kp_to_matches1 = []
    pct_kp_to_matches2 = []
    pct_m_after_ransac = []
    descriptor_time = []
    matcher_time = []
    time_per_desc = []
    time_per_match = []
    intersect_h = []
    intersect_m = []
    intersect_a = []
    correl_h = []
    correl_m = []
    correl_a = []
    chsq_h = []
    chsq_m = []
    chsq_a = []
    euc_h = []
    euc_m = []
    euc_a = []
    cheb_h = []
    cheb_m = []
    cheb_a = []
    resized = []
    
    for i in range(batch_df.shape[0]):
        row = batch_df.iloc[i]
        resized.append(row['resized'])
        ma2, r_e, t_e = get_row_score(row)
        img1 = cv2.imread(row['img1'])
        img2 = cv2.imread(row['img2'])
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        
        
        kps1 = len(list(row['kp1'][0]))
        kps2 = len(list(row['kp2'][0]))
        m_b = len(row['MatchIdxBefore'])
        m_a = len(row['MatchIdxAfter'])
        pct_kp_m1 = m_b / kps1
        pct_kp_m2 = m_b / kps2
        pct_m_a = m_a / m_b
        d_t = row['desc_time']
        m_t = row['match_time']
        tpd = d_t / (kps1 + kps2) 
        tpm = m_t / m_b
        
        descriptor_name.append(descriptorName)
        pretrained.append(pre_trained)
        scene.append(row['scene'])
        maa.append(ma2)
        r_error.append(r_e)
        t_error.append(t_e)
        
        covis.append(row['covis'])
        size_dif.append(abs(1 - ((img1.shape[0] * img1.shape[1]) / (img2.shape[0] * img2.shape[1]))))
        height1.append(img1.shape[0])
        width1.append(img1.shape[1])
        height2.append(img2.shape[0])
        width2.append(img2.shape[1])
        keypoints1.append(kps1)
        keypoints2.append(kps2)
        matches_before.append(m_b)
        matches_after.append(m_a)
        pct_kp_to_matches1.append(pct_kp_m1)
        pct_kp_to_matches2.append(pct_kp_m2)
        pct_m_after_ransac.append(pct_m_a)
        descriptor_time.append(d_t)
        matcher_time.append(m_t)
        time_per_desc.append(tpd)
        time_per_match.append(tpm)
        
        hist1 = cv2.calcHist([img1], [0], None, [16], [0, 256])
        hist1 = hist1 / hist1.sum()
        hist2 = cv2.calcHist([img2], [0], None, [16], [0, 256])
        hist2 = hist2 / hist2.sum()
        
        gx1 = cv2.Sobel(img1, cv2.CV_32F, 1, 0, ksize=1)
        gy1 = cv2.Sobel(img1, cv2.CV_32F, 0, 1, ksize=1)
        mag1, angle1 = cv2.cartToPolar(gx1, gy1, angleInDegrees=True)
        gx2 = cv2.Sobel(img2, cv2.CV_32F, 1, 0, ksize=1)
        gy2 = cv2.Sobel(img2, cv2.CV_32F, 0, 1, ksize=1)
        mag2, angle2 = cv2.cartToPolar(gx2, gy2, angleInDegrees=True)
        
        mhist1 = cv2.calcHist([mag1], [0], None, [16], [0, 256])
        mhist1 = mhist1 / mhist1.sum()
        mhist2 = cv2.calcHist([mag2], [0], None, [16], [0, 256])
        mhist2 = mhist2 / mhist2.sum()
        ahist1 = cv2.calcHist([angle1], [0], None, [16], [0, 360])
        ahist1 = ahist1 / ahist1.sum()
        ahist2 = cv2.calcHist([angle2], [0], None, [16], [0, 360])
        ahist2 = ahist2 / ahist2.sum()
        
        intersect_h.append(cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT))
        intersect_m.append(cv2.compareHist(mhist1, mhist2, cv2.HISTCMP_INTERSECT))
        intersect_a.append(cv2.compareHist(ahist1, ahist2, cv2.HISTCMP_INTERSECT))

        correl_h.append(cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL))
        correl_m.append(cv2.compareHist(mhist1, mhist2, cv2.HISTCMP_CORREL))
        correl_a.append(cv2.compareHist(ahist1, ahist2, cv2.HISTCMP_CORREL))
        
        chsq_h.append(cv2.compareHist(hist1, hist2, cv2.HISTCMP_CHISQR))
        chsq_m.append(cv2.compareHist(mhist1, mhist2, cv2.HISTCMP_CHISQR_ALT))
        chsq_a.append(cv2.compareHist(ahist1, ahist2, cv2.HISTCMP_CHISQR))
        euc_h.append(dist.euclidean(hist1, hist2))
        euc_m.append(dist.euclidean(mhist1, mhist2))
        euc_a.append(dist.euclidean(ahist1, ahist2))
        cheb_h.append(dist.chebyshev(hist1, hist2))
        cheb_m.append(dist.chebyshev(mhist1, mhist2))
        cheb_a.append(dist.chebyshev(ahist1, ahist2))

        
    df = pd.DataFrame({'descriptor' : descriptor_name, 'resized' : resized, 'pretrained' : pretrained, 'scene': scene,
                      'maa': maa, 'rotation error' : r_error, 'translation error' : t_error,
                      'covisibility' : covis, 'size dif' : size_dif,
                       'width 1' : width1, 'height 1' : height1, 'width 2' : width2, 'height 2':height2, 
                      'keypoints found 1' : keypoints1, 'keypoints found 2' : keypoints2,
                      'matches before' : matches_before, 'matches after' : matches_after, 
                       '% KP as matches 1' : pct_kp_to_matches1, '% KP as matches 2' : pct_kp_to_matches2, 
                       '% matches after ransac' : pct_m_after_ransac,
                       'descriptor time' : descriptor_time, 'matcher time' : matcher_time,
                      'time per desc' : time_per_desc, 'time per match' : time_per_match,
                      'inter': intersect_h, 'inter m': intersect_m, 'inter a' : intersect_a,
                      'cor':correl_h, 'cor m' : correl_m, 'cor a':correl_a, 'chsq' : chsq_h,
                      'chsq m' : chsq_m, 'chsq a':chsq_a, 'euc' : euc_h, 'euc m':euc_m,'euc a':euc_a,
                      'cheb':cheb_h, 'cheb m':cheb_m, 'cheb a' : cheb_a})
    if overwrite:
        df.to_csv('/kaggle/working/model_stats.csv', mode='w')
    else:  
        df.to_csv('/kaggle/working/model_stats.csv', header=False, mode='a')
        
    del df,descriptor_name,pretrained,scene,maa,r_error,t_error,covis,size_dif,width1,width2,height1,height2,keypoints1,keypoints2,matches_before,matches_after,pct_kp_to_matches1,pct_kp_to_matches2,pct_m_after_ransac,descriptor_time,matcher_time,time_per_desc,time_per_match,intersect_h,intersect_m,intersect_a,correl_h,correl_m,correl_a,chsq_h,chsq_m,chsq_a,euc_h,euc_m,euc_a,cheb_h,cheb_m,cheb_a,resized
    gc.collect()
    torch.cuda.empty_cache()
    

In [ ]:
# Runs a matching process for a batch of lafs given a kornia descriptor
def run_test(lafs, descriptor, resize=False):
    descriptors = get_batch_descriptors(lafs, descriptor, resize)
    torch.cuda.empty_cache()
    matches = get_batch_matches(descriptors)
    torch.cuda.empty_cache()
    get_score(matches)
    return matches
    del descriptors, matches; gc.collect()
    torch.cuda.empty_cache()

In [ ]:
# Descriptors
# Loads the kornia descriptors that will be used
SIFT = KF.SIFTDescriptor(patch_size=32, num_ang_bins=8, num_spatial_bins=4, rootsift=True, clipval=0.2).eval().to(device)
MKD = KF.MKDDescriptor(patch_size=32, kernel_type='concat', whitening='pcawt', training_set='liberty', output_dims=128).eval().to(device)
HardNet = KF.HardNet(True).eval().to(device)
HardNet8 = KF.HardNet8(True).eval().to(device)
try: 
    HyNet = KF.hynet.HyNet(pretrained=True, is_bias=True, is_bias_FRN=True, dim_desc=128, drop_rate=0.3, eps_l2_norm=1e-10).eval().to(device)
except:
    HyNet = HyNet(pretrained=True, is_bias=True, is_bias_FRN=True, dim_desc=128, drop_rate=0.3, eps_l2_norm=1e-10).eval().to(device)
TFeat = KF.TFeat(True).eval().to(device)
SOSNet = KF.SOSNet(True).eval().to(device)


In [ ]:
%%time
# Loading a batch of image pairs
# 8 images per scene for a total of 128 images in batch seems to be optimal batch size
# There is little performance penalty vs 4 images per scene, but >8 uses too much memory

# Getting LAFs for image pairs
# Preloading LAFs saves time and memory
p_b = get_pair_batch(8, 0.1)
lafs = get_batch_lafs(p_b)

In [ ]:
%%time
# Runs a matching process with SIFT descriptor
# This could be considered the baseline
sift = run_test(lafs, SIFT)
stats_to_csv("SIFT", "True", sift, overwrite=True)
display_example_from_batch(sift)
del sift
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# MKD kept breaking the GPU so it is commented out 

# %%time
# mkd = run_test(lafs, MKD)
# stats_to_csv("MKD", "True", mkd, overwrite=False)
# del mkd
# gc.collect()

In [ ]:
%%time
# Runs a matching process with HardNet descriptor
hardnet = run_test(lafs, HardNet)
stats_to_csv("HardNet", "True", hardnet, overwrite=False)
display_example_from_batch(hardnet)
del hardnet
gc.collect()

In [ ]:
%%time
# Runs a matching proces with HardNet8 descriptor
hardnet8 = run_test(lafs, HardNet8)
stats_to_csv("HardNet8", "True", hardnet8, overwrite=False)
display_example_from_batch(hardnet8)
del hardnet8
gc.collect()

In [ ]:
%%time
# Runs a matching proces with HyNet descriptor
hynet = run_test(lafs, HyNet)
stats_to_csv("HyNet", "True", hynet, overwrite=False)
display_example_from_batch(hynet)
del hynet
gc.collect()

In [ ]:
%%time
# Runs a matching proces with TFeat descriptor
tfeat = run_test(lafs, TFeat)
stats_to_csv("TFeat", "True", tfeat, overwrite=False)
display_example_from_batch(tfeat)
del tfeat
gc.collect()

In [ ]:
%%time
# Runs a matching proces with SOSNet descriptor
sosnet = run_test(lafs, SOSNet)
stats_to_csv("SOSNet", "True", sosnet, overwrite=False)
display_example_from_batch(sosnet)
del sosnet
gc.collect()

In [ ]:
%%time
# Runs a matching proces with LoFTR matcher
# This could also be considered a baseline "to beat" because it is consistently highest performing
# Goal of project is finding edge cases where other descriptors might be better and/or ways to tweak LoFTR
loftra = loftr_match(lafs, LOFTR_A)
stats_to_csv("LoFTR A", "True", loftra, overwrite=False)
display_example_from_batch(loftra, loftr=True)
del loftra
gc.collect()

<a href="/kaggle/working/model_stats.csv"> Download File </a>

## 